# Bibliotecas

In [57]:
import pandas as pd

# Limpeza dos dados

## Lendo os arquivos

In [58]:
df_americanas = pd.read_csv(f"Dados/produtos_americanas.csv", dtype = {'ean': str})
df_amazon = pd.read_csv(f"Dados/produtos_amazon.csv", dtype = {'ean': str})

In [59]:
df_amazon["ean"].value_counts()

0713477350430    57
9095247871972    51
7908414420304     2
7319625661149     1
7465214678507     1
                 ..
0708570152586     1
8263430934112     1
6973553520094     1
3041412361256     1
6934177770012     1
Name: ean, Length: 1017, dtype: int64

## Retirando títulos com palavras indesejadas

In [60]:
bad_words = ["acessório", "acessorio", "adaptador", "alargadores", "anel", "arduino", "atomiser", "avatar",
             "aparelho de som", "automotivo", "axiom",
             "bandeja", "biometria", "bike", "boroscópio", "box", 
             "cable", "cabo", "caixa", "campainha", "calculator", "calculadora", "capa", "capinha", "card",
             "case", "caso", "carregador", "carro"
             "chassi", "chaveiro", "chave seletora", "clip",
             "coletor", "collar", "conjunto", "comoda", "cômoda", "cordão", "cortador", "cosplay",
             "drone", "eon", "eixo", "eixos", "espelho", "filme", "franquia", "gancho", "grânulos", "gaveta",
             "handmade", "hemobblo", "hemobllo", "hd externo", "jbl", "junta", "kit",
             "lente", "lixa", "lurrose",
             "m-disc", "mágico", "magico", "mesa", "microsd", "monitor", "montagem", "moochy",
             "netcna", "óculos", "oculos", "ornament",
             "papel", "parede", "part", "partes", "pedal", "pen drive", "pendrive", "pneu", "pneus", "power bank",
             "prato", "prettyzoom", "pretyzoom", "proteção", "protetor", "pulseira",
             "raspberry", "radiador", "rank de memória", "realidade virtual", "refrigerador", "regeneradora",
             "relogio", "relógio", "remoção", "reparo", "reposição", "reposiçao", "resfriador", "retrovisor", "roda",
             "ronyme", "rv", "scanner", "sensor", "smart tv", "soquete", "stick", "streaming box", "suporte",
             "tablet", "tampa", "tomada", "tomadas", "tradutor", "turbopower",
             "ultra-sensível", "upgrade", "upgrades",
             "vador", "veado", "vedação", "verbatim", "vidro", "vr", "xilofone", "xylophone",
             "watch", "zawa", "zbox"
            ]

print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

df_amazon['titulo_cb'] = df_amazon['titulo'].str.lower()
df_americanas['titulo_cb'] = df_americanas['titulo'].str.lower()

df_amazon_ft = df_amazon[~df_amazon["titulo_cb"].str.contains('|'.join(bad_words))]
df_americanas_ft = df_americanas[~df_americanas["titulo_cb"].str.contains('|'.join(bad_words))]
print(f"\nFiltrado:\nAmazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (4333, 4)
Americanas: (1032, 4)

Filtrado:
Amazon: (1911, 5)
Americanas: (876, 5)


In [61]:
#df_amazon_ft['url'].loc[970]

In [62]:
#df_amazon_ft[350:].head(50)

## Encontrando EANs repetidos na mesma loja

Removendo produtos com o mesmo título

In [63]:
df_amazon_ft = df_amazon_ft.loc[~df_amazon_ft["titulo"].duplicated(keep = 'first')]
df_americanas_ft = df_americanas_ft.loc[~df_americanas_ft["titulo"].duplicated(keep = 'first')]

In [64]:
print(f"Amazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

Amazon: (1738, 5)
Americanas: (815, 5)


In [85]:
df_americanas_ft[df_americanas_ft["ean"].notnull()].shape

(508, 6)

In [87]:
df_amazon_ft[df_amazon_ft["ean"].notnull()].shape

(467, 6)

Verificando Títulos de EANs duplicados

In [65]:
ama_vc_ean = df_amazon_ft[df_amazon_ft["ean"].notnull()]["ean"].value_counts()
ama_duplicados = ama_vc_ean[ama_vc_ean > 1].index[0]
df_amazon_ft[df_amazon_ft['ean'] == ama_duplicados]

,url,titulo,ean,descricao,titulo_cb
262,https://www.amazon.com.br/Smartphone-Multilase...,Smartphone Multilaser G Max 2 2GB + 64GB Tela ...,7908414420304,Sobre este item Câmera Tripla 8MP + VGA + V...,smartphone multilaser g max 2 2gb + 64gb tela ...
2632,https://www.amazon.com.br/Smartphone-Multilase...,Smartphone Multilaser G Max 2 Dourado 64GB 2GB...,7908414420304,Sobre este item Câmera Tripla 8MP + VGA + V...,smartphone multilaser g max 2 dourado 64gb 2gb...


In [66]:
ame_vc_ean = df_americanas_ft[df_americanas_ft["ean"].notnull()]["ean"].value_counts()
ame_duplicados = ame_vc_ean[ame_vc_ean > 1].index
df_americanas_ft[df_americanas_ft["ean"].isin(ame_duplicados)].sort_values(by = ['ean']).head(1)

,descricao,ean,titulo,url,titulo_cb
459,Características gerais Redmi Modelo - Note 10 ...,6934177734519,Redmi Note 10 Pro Dual Sim 128 gb 6 Gb Azul,https://www.americanas.com.br/produto/4431267188,redmi note 10 pro dual sim 128 gb 6 gb azul


In [67]:
df_americanas_ft['loja'] = "americanas"
df_amazon_ft['loja'] = "amazon"

In [68]:
frames = [df_americanas_ft, df_amazon_ft]
df_final = pd.concat(frames, ignore_index = True)

In [69]:
vc_ean = df_final[df_final["ean"].notnull()]["ean"].value_counts()
duplicados = vc_ean[vc_ean > 1].index
df_final_dup = df_final[df_final["ean"].isin(duplicados)].sort_values(by = ['ean'])

## Fazendo matches

In [70]:
def criar_dicionario(descricao_1, ean_1, titulo_1, url_1, titulo_cb_1, loja_1,
                     descricao_2, ean_2, titulo_2, url_2, titulo_cb_2, loja_2,
                     match):
    
    return {'descricao_1' : descricao_1 , 'ean_1' : ean_1, 'titulo_1' : titulo_1,
            'url_1' : url_1, 'titulo_cb_1' : titulo_cb_1, 'loja_1' : loja_1,
            
            'descricao_2' : descricao_2 , 'ean_2' : ean_2, 'titulo_2' : titulo_2,
            'url_2' : url_2 , 'titulo_cb_2' : titulo_cb_2, 'loja_2' : loja_2,
            
            'match' : match
           }

In [71]:
def criar_pares(colunas, df_final_dup):

    df_pares = pd.DataFrame(columns = colunas)
    df_pares

    lista_ean = []

    for i in range(len(df_final_dup)):
        
        if df_final_dup.iloc[i]['ean'] not in lista_ean:
            
            lista_ean.append(df_final_dup.iloc[i]['ean'])
            
            indice = df_final_dup.index[df_final_dup['ean'] == df_final_dup.iloc[i]['ean']].tolist()
            
            for j in range(len(indice)-1):
                
                for k in range(j + 1, len(indice)):
                    
                    dicionario = criar_dicionario(descricao_1  = df_final_dup.loc[indice[j]]['descricao'] , ean_1 = df_final_dup.loc[indice[j]]['ean'],
                                                  titulo_1 = df_final_dup.loc[indice[j]]['titulo'], url_1 = df_final_dup.loc[indice[j]]['url'],
                                                  titulo_cb_1 = df_final_dup.loc[indice[j]]['titulo_cb'], loja_1 = df_final_dup.loc[indice[j]]['loja'],
                                                  
                                                  descricao_2 = df_final_dup.loc[indice[k]]['descricao'] , ean_2 = df_final_dup.loc[indice[k]]['ean'],
                                                  titulo_2 = df_final_dup.loc[indice[k]]['titulo'], url_2 = df_final_dup.loc[indice[k]]['url'],
                                                  titulo_cb_2 = df_final_dup.loc[indice[k]]['titulo_cb'], loja_2 = df_final_dup.loc[indice[k]]['loja'],
                                                  
                                                  match = 1
                                                  )
                
                    df_pares = df_pares.append(dicionario, ignore_index = True)

    return df_pares

In [72]:
colunas = ["descricao_1", "ean_1", "titulo_1", "url_1",	"titulo_cb_1", "loja_1",
           "descricao_2", "ean_2", "titulo_2", "url_2",	"titulo_cb_2", "loja_2",
           "match"]

In [73]:
df_pares = criar_pares(colunas, df_final_dup)

In [74]:
#df_pares[df_pares['ean_1']=="6934177738180"][['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]
#df_pares[['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]

## Salvando Pares

In [75]:
#df_pares.to_csv(f"Dados/pares_smartphones.csv", index = False)

In [76]:
df_pares = pd.read_csv("Dados/pares_smartphones.csv")

## Criando pares que não são matches

In [77]:
df_final_ean = df_final[df_final["ean"].notnull()]
df_final_ean.shape

(975, 6)

In [78]:
n_matches = df_pares.shape[0]*5
df_nao_pares = pd.DataFrame(columns = colunas)

for i in range(n_matches):
    created_match = False
    
    while(not created_match):
        df_sorteado = df_final_ean.sample(n = 2)
        if(df_sorteado.iloc[0]['ean']!= df_sorteado.iloc[1]['ean']):
            
            dic = criar_dicionario(df_sorteado.iloc[0]['descricao'], df_sorteado.iloc[0]['ean'], df_sorteado.iloc[0]['titulo'], df_sorteado.iloc[0]['url'], df_sorteado.iloc[0]['titulo_cb'], df_sorteado.iloc[0]['loja'],
                                   df_sorteado.iloc[1]['descricao'], df_sorteado.iloc[1]['ean'], df_sorteado.iloc[1]['titulo'], df_sorteado.iloc[1]['url'], df_sorteado.iloc[1]['titulo_cb'], df_sorteado.iloc[1]['loja'],
                                   0)
            
            df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
            created_match = True

Salvando pares não matches

In [79]:
df_nao_pares.to_csv(f"Dados/nao_pares_smartphones.csv", index = False)

## Gerando dataset final

In [80]:
df_pares = pd.read_csv("Dados/pares_smartphones.csv")
df_nao_pares = pd.read_csv("Dados/nao_pares_smartphones.csv")

df_pares_total = pd.concat([df_pares, df_nao_pares], ignore_index = True)

In [81]:
df_pares_total = df_pares_total.sample(frac = 1)
df_pares_total.to_csv(f"Dados/total_pares_smartphones.csv", index = False)

In [82]:
df_pares_total

,descricao_1,ean_1,titulo_1,url_1,titulo_cb_1,loja_1,descricao_2,ean_2,titulo_2,url_2,titulo_cb_2,loja_2,match
294,Todas as informações divulgadas são de respons...,7892509113830,Smartphone Samsung Galaxy S20+ 128GB 4G Wi-Fi ...,https://www.americanas.com.br/produto/1524131271,smartphone samsung galaxy s20+ 128gb 4g wi-fi ...,americanas,Sobre este item Esta câmera inovadora é ade...,715076307266,"Smartphone com câmera dupla HD de 6,7 polegada...",https://www.amazon.com.br/dp/B09ZL914SW/ref=sr...,"smartphone com câmera dupla hd de 6,7 polegada...",amazon,0
234,Características gerais Redmi Modelo - Note 10 ...,6934177734519,Redmi Note 10 Pro Dual Sim 128 gb 6 Gb Azul,https://www.americanas.com.br/produto/4431267188,redmi note 10 pro dual sim 128 gb 6 gb azul,americanas,Smartphone Samsung Galaxy A22 128gb 6gb Ram Te...,8806092643338,Smartphone Samsung Galaxy A22 128gb 6gb Ram Te...,https://www.americanas.com.br/produto/3932108943,smartphone samsung galaxy a22 128gb 6gb ram te...,americanas,0
69,O rog Phone 5 é o smartphone gamer com process...,7898573297896,Smartphone asus rog Phone 5 8GB/128GB Preto,https://www.americanas.com.br/produto/3954396245,smartphone asus rog phone 5 8gb/128gb preto,americanas,NaN,7898573297896,Smartphone ASUS ROG Phone 5 8GB/ 128GB Preto,https://www.amazon.com.br/Smartphone-ASUS-Phon...,smartphone asus rog phone 5 8gb/ 128gb preto,amazon,1
56,Todas as informações divulgadas são de respons...,7892597351657,Smartphone Motorola Moto G31 128GB 4G Wi-Fi Te...,https://www.americanas.com.br/produto/4447206291,smartphone motorola moto g31 128gb 4g wi-fi te...,americanas,Sobre este item Cores surreais na tela OLED...,7892597351657,Smartphone Motorola Moto G31 128GB 4GB RAM Azul,https://www.amazon.com.br/Smartphone-Motorola-...,smartphone motorola moto g31 128gb 4gb ram azul,amazon,1
308,Tecnologia e conectividade com o Smartphone LG...,7893299914997,Smartphone LG K22+ 64GB 4G Wi-Fi Tela 6.2'' Du...,https://www.americanas.com.br/produto/1965288493,smartphone lg k22+ 64gb 4g wi-fi tela 6.2'' du...,americanas,Smartphone Asus Zenfone Max Shot 32gb/3gb 4gb ...,7898573296479,Smartphone Asus Zenfone Max Shot 32gb/3gb 4gb ...,https://www.americanas.com.br/produto/2634939466,smartphone asus zenfone max shot 32gb/3gb 4gb ...,americanas,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,Sobre este item Habilitado 5G ：S22 Ultra pe...,1366844004970,S22 Ultra 5G Celular Smartphone Tela Full HD D...,https://www.amazon.com.br/Celular-Smartphone-P...,s22 ultra 5g celular smartphone tela full hd d...,amazon,Sobre este item Processador poderoso: O pro...,7605066440970,"Smartphone com tela de 4,66 polegadas, Smartph...",https://www.amazon.com.br/Smartphone-polegadas...,"smartphone com tela de 4,66 polegadas, smartph...",amazon,0
281,O Zenfone Max M2 é um Smartphone completo com ...,7898573296363,Smartphone Zenfone Asus Max M2 ZB555KL-4A158BR...,https://www.americanas.com.br/produto/134218822,smartphone zenfone asus max m2 zb555kl-4a158br...,americanas,NaN,7892597349067,Smartphone Motorola One Hyper 128GB 4G Wi-Fi T...,https://www.americanas.com.br/produto/1348709660,smartphone motorola one hyper 128gb 4g wi-fi t...,americanas,0
339,Sobre este item \r\n Isso se adeque às suas ...,5012786044210,"Motorola Baba Eletronica Ease34, Motorola, Branco",https://www.amazon.com.br/Motorola-Baba-Eletro...,"motorola baba eletronica ease34, motorola, branco",amazon,Sobre este item ❤️ 【Tela de reconhecimento ...,4394085359075,Celular A60 Pro 4G com tela de gota d'água de ...,https://www.amazon.com.br/Celular-polegadas-Sm...,celular a60 pro 4g com tela de gota d'água de ...,amazon,0
148,O Zenfone 8 Flip foi construído para desafiar ...,4711081359845,Smartphone asus Zenfone 8 Flip 8GB/128GB Preto,https://www.americanas.com.br/produto/4266425516,smartphone asus zenfone 8 flip 8gb/128gb preto,americanas,"Smartphone Multilaser G Max Preto, tenha o mel...",7899838865461,Smartphone multilaser G max 4G 32GB tela 6 1GB...,https://www.americanas.